# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. ETL pipeline for Apache Cassandra

## Now we are are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
# Creating a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project2 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('project2')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. 

## Our goal is to create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### QUERY 1

In [9]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

#Lets make both sessionId and itemInSession are part of the PARTITION KEY to make it unique

query = "CREATE TABLE IF NOT EXISTS songs_library"
query = query + " (sessionId INT, itemInSession INT, artist TEXT, song_title TEXT, \
                song_length FLOAT, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [10]:
# Code to set up the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## INSERT statements into the `query` variable
        query = "INSERT INTO songs_library (sessionId, itemInSession, artist, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## Set up column elements shouldfor each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, we would change the code below to `line[0], line[1]`
        ## from the table event_datafile_new.csv 
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [11]:
## SELECT statement to verify the data was entered into the table for query 1
query = "SELECT artist, song_title, song_length FROM songs_library \
        WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as error:
    print(error)
for row in rows:
    print(row.artist, "|", row.song_title, "|", row.song_length)

Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875


### QUERY 2

In [ ]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

# Both userId and sessionId are part of the PARTITION KEY and itemInSession as CLUSTERED COLUMN
# because we need the result sortered by itemInSession.
query = "CREATE TABLE IF NOT EXISTS user_sessions"
query = query + " (userId INT, sessionId INT, itemInSession INT, artist TEXT, song_title TEXT, \
            user_first_name TEXT, user_last_name TEXT, PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)


In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## INSERT statements into the `query` variable
        query = "INSERT INTO user_sessions (userId, sessionId, itemInSession, artist, song_title, \
                user_first_name, user_last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [ ]:
## SELECT statement to verify the data was entered into the table for query 2
query = "SELECT artist, song_title, user_first_name, user_last_name FROM user_sessions \
        WHERE userId = 10 AND sessionId = 182"
try:
    rows = session.execute(query)
except Exception as error:
    print(error)
for row in rows:
    print(row.artist, "|", row.song_title, "|", row.user_first_name, "|", row.user_last_name)

### QUERY 3

In [ ]:
## Query 3: Give me every user name (first and last) in my music app history 
## who listened to the song 'All Hands Against His Own'

#song_title and userId are the COMPOSITE KEY to make it unique
query = "CREATE TABLE IF NOT EXISTS app_logs"
query = query + " (song_title TEXT, user_first_name TEXT, \
                    user_last_name TEXT, userId INT, PRIMARY KEY(song_title, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## INSERT statements into the `query` variable
        query = "INSERT INTO app_logs (song_title, user_first_name, user_last_name, userId)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], line[1], line[4], int(line[10])))

In [ ]:
## SELECT statement to verify the data was entered into the table for query 3
query = "SELECT user_first_name, user_last_name FROM app_logs \
        WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as error:
    print(error)
for row in rows:
    print(row.user_first_name, "|", row.user_last_name)

### Drop the tables before closing out the sessions

In [ ]:
## Drop the table before closing out the sessions
try:
    session.execute("DROP TABLE IF EXISTS songs_library")
    session.execute("DROP TABLE IF EXISTS user_sessions")
    session.execute("DROP TABLE IF EXISTS app_logs")
except Exception as error:
    print(error)  

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()